<a href="https://colab.research.google.com/github/hiteshsurya17/hiteshsurya17-INFO-5731-Section-020---Computational-Methods-for-Information-Systems-Fall-2024-1-/blob/main/INFO5731_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 2**

The purpose of this exercise is to understand users' information needs, and then collect data from different sources for analysis by implementing web scraping using Python.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission. , and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)
Describe an interesting research question (or practical question or something innovative) you have in mind, what kind of data should be collected to answer the question(s)? Specify the amount of data needed for analysis. Provide detailed steps for collecting and saving the data.

In [7]:
# The sentiment of news articles about tech companies impact their stock prices , I want to find out how is this affected on the company stock?
# Data to Collect:
# News Articles: Full text of news articles related to technology companies. Date and time of publication, source of the article.
# Collect articles for a minimum of 1 year ,at least 500-1000 articles per company per year .
# Stock Prices: Historical stock price data for technology companies. Date and time of each stock price record.
# Collect daily stock price data(opening price, closing price, highest price, and lowest price for each trading day) for the same period as the news articles.

## Question 2 (10 Points)
Write Python code to collect a dataset of 1000 samples related to the question discussed in Question 1.

In [27]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_news_articles():
    url = "https://www.theverge.com/tech"
    article_list = []


    for page in range(1, 101):
        response = requests.get(f"{url}/page/{page}")
        soup = BeautifulSoup(response.text, 'html.parser')

        articles = soup.find_all('article')
        for article in articles:
            title = article.find('h2').text
            content = article.find('p').text
            publication_date = article.find('time')['datetime']
            article_list.append([title, content, publication_date])

            if len(article_list) >= 1000:
                break
        if len(article_list) >= 1000:
            break

    with open('news_articles.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title', 'Content', 'Publication Date'])
        writer.writerows(article_list)

    print("Collected 1000 news articles and saved to 'news_articles.csv'.")

scrape_news_articles()


Collected 1000 news articles and saved to 'news_articles.csv'.


## Question 3 (10 Points)
Write Python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "XYZ". The articles should be published in the last 10 years (2014-2024).

The following information from the article needs to be collected:

(1) Title of the article

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [40]:
pip install requests pandas


In [43]:
from scholarly import scholarly
import pandas as pd
import time

def fetch_google_scholar_articles(query, year_start, year_end, max_results=1000):
    all_articles = []
    search_query = scholarly.search_pubs(query)

    while len(all_articles) < max_results:
        try:
            paper = next(search_query)
            publication_year = paper.get('bib', {}).get('pub_year', 'N/A')
            if publication_year.isdigit() and int(publication_year) >= year_start and int(publication_year) <= year_end:
                article = {
                    'Title': paper.get('bib', {}).get('title', 'N/A'),
                    'Venue/Journal/Conference': paper.get('bib', {}).get('journal', 'N/A'),
                    'Year': publication_year,
                    'Authors': ', '.join(paper.get('bib', {}).get('author', [])),
                    'Abstract': paper.get('bib', {}).get('abstract', 'N/A')
                }
                all_articles.append(article)

                if len(all_articles) >= max_results:
                    break

            time.sleep(1)
        except StopIteration:
            break

    return all_articles

def save_to_csv(articles, filename='articles.csv'):
    df = pd.DataFrame(articles)
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"Saved {len(articles)} articles to '{filename}'.")

articles = fetch_google_scholar_articles('XYZ', 2014, 2024)
save_to_csv(articles)


MaxTriesExceededException: Cannot Fetch from Google Scholar.

## Question 4A (10 Points)
Develop Python code to collect data from social media platforms like Reddit, Instagram, Twitter (formerly known as X), Facebook, or any other. Use hashtags, keywords, usernames, or user IDs to gather the data.



Ensure that the collected data has more than four columns.


In [32]:
pip install tweepy pandas


In [42]:
import tweepy
import pandas as pd

# Replace these with your actual API keys
API_KEY = 'your_api_key'
API_SECRET_KEY = 'your_api_secret_key'
ACCESS_TOKEN = 'your_access_token'
ACCESS_TOKEN_SECRET = 'your_access_token_secret'

# Authenticate to Twitter
auth = tweepy.OAuth1UserHandler(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

def collect_tweets(query, count=100):
    """Collect tweets based on the query."""
    tweets = []
    for tweet in tweepy.Cursor(api.search_tweets, q=query, lang='en', tweet_mode='extended').items(count):
        tweets.append({
            'tweet_id': tweet.id_str,
            'created_at': tweet.created_at,
            'username': tweet.user.screen_name,
            'followers_count': tweet.user.followers_count,
            'retweets': tweet.retweet_count,
            'favorites': tweet.favorite_count,
            'text': tweet.full_text
        })
    return tweets

def save_to_csv(data, filename):
    """Save data to a CSV file."""
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding='utf-8')

def main():
    query = '#example'  # Replace with your desired hashtag or keyword
    tweets = collect_tweets(query, count=1000)
    save_to_csv(tweets, 'twitter_data.csv')
    print(f'Successfully collected and saved {len(tweets)} tweets.')

if __name__ == '__main__':
    main()


ConnectionError: HTTPSConnectionPool(host='api.labs.cognitive.microsoft.com', port=443): Max retries exceeded with url: /academic/v1.0/evaluate?expr=Ti%3D%27XYZ%27&count=100&offset=0&orderby=Year+desc&attributes=Ti%2CCC%2CY%2CAA%2CAb&filters=Y%3E%3D+2014+AND+Y%3C%3D+2024 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7961b2027bb0>: Failed to resolve 'api.labs.cognitive.microsoft.com' ([Errno -2] Name or service not known)"))

## Question 4B (10 Points)
If you encounter challenges with Question-4 web scraping using Python, employ any online tools such as ParseHub or Octoparse for data extraction. Introduce the selected tool, outline the steps for web scraping, and showcase the final output in formats like CSV or Excel.



Upload a document (Word or PDF File) in any shared storage (preferably UNT OneDrive) and add the publicly accessible link in the below code cell.

Please only choose one option for question 4. If you do both options, we will grade only the first one

In [ ]:
# write your answer here


# Mandatory Question

**Important: Reflective Feedback on Web Scraping and Data Collection**



Please share your thoughts and feedback on the web scraping and data collection exercises you have completed in this assignment. Consider the following points in your response:



Learning Experience: Describe your overall learning experience in working on web scraping tasks. What were the key concepts or techniques you found most beneficial in understanding the process of extracting data from various online sources?



Challenges Encountered: Were there specific difficulties in collecting data from certain websites, and how did you overcome them? If you opted for the non-coding option, share your experience with the chosen tool.



Relevance to Your Field of Study: How might the ability to gather and analyze data from online sources enhance your work or research?

**(no grading of your submission if this question is left unanswered)**

In [ ]:
'''
Write your response here.
'''